In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import re
import json

In [2]:
path = "G:/Research_projects/pennmutual/range_reasoning/data_augmentation/test_all_option_infer_output.csv"
df = pd.read_csv(path)

# Make Data

Format 1. Our format

e.g.

Q: Given the following range definitions: Range 1: [39, 48], Range 2: [49, 58], Range 3: [59, 68]. Please identify the range for the number 61 and explain your reasoning. Provide the name of the range at the end.

A: range 3. Here's how I determined this: First, I enumerated all the values in each range with 1 decimal point precision: Range 1: [39, 40, 41, 42, 43, 44, 45, 46, 47, 48], Range 2: [49, 50, 51, 52, 53, 54, 55, 56, 57, 58], Range 3: [59, 60, 61, 62, 63, 64, 65, 66, 67, 68]. Then, I checked if the given number (61) is in each of these sets. It is not in Range 1 or Range 2, but it is in Range 3. Therefore, the number 61 falls within Range 3. The name of the range at the end is: Range 3.


In [21]:
random.randint(0,2)

1

In [35]:
a = [1,2,3,4,5,6,7,8,9,10]
a[-2:]

[9, 10]

In [61]:
score_1 = []
score_2 = []
score_3 = []
for i, d in df.iterrows():
    
    sample_id = d['sample_id']
    number = d['number']
    range_names = re.findall(r"'(.*?)'", d['Ranges'])
    Range_1_name = range_names[0].replace("_", " ")
    Range_2_name = range_names[1].replace("_", " ")
    Range_3_name = range_names[2].replace("_", " ")
    ranges = []
    for ran in re.findall(r'\[(.*?)\]', d['Ranges']):
        ranges.append(ran.split(", "))
    #print(ranges[0])
    if float(ranges[0][1]) - float(ranges[0][0]) < 1:
        gap = 1
    else:
        gap = 10
        
    start = float(ranges[0][0])
    
    Range_1_enum = []
    
    if gap == 1:
        deno = 10
        for g in map(lambda x: x / deno, range(0, 10)):
            Range_1_enum.append(start+g)
            Range_1 = [start, Range_1_enum[-1]]
            #print(Range_1)

            Range_2_enum = [x+gap for x in Range_1_enum]
            Range_2 = [start+gap, Range_2_enum[-1]]
            #print(Range_2)

            Range_3_enum = [x+gap for x in Range_2_enum]
            Range_3 = [start+2*gap, Range_3_enum[-1]]
            #print(Range_3)
    else:
        deno = 1
        start = int(start)
        for g in map(lambda x: x / deno, range(0, 10)):
            Range_1_enum.append(int(start+g))
            Range_1 = [start, Range_1_enum[-1]]
            #print(Range_1)

            Range_2_enum = [int(x+gap) for x in Range_1_enum]
            Range_2 = [start+gap, Range_2_enum[-1]]
            #print(Range_2)

            Range_3_enum = [int(x+gap) for x in Range_2_enum]
            Range_3 = [start+2*gap, Range_3_enum[-1]]
            #print(Range_3)
        
    #print(ranges[0])
    #print(Range_1_enum)
    
    # score 2
    temp_2 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " +  f"Here's how I determined this: First, I enumerated all the values in each range with 1 decimal point precision: {Range_1_name}: {Range_1_enum}, {Range_2_name}: {Range_2_enum}, {Range_3_name}: {Range_3_enum}."
    score_2.append([sample_id, temp_2, 1])
    #print(temp_2)
    
    wrong_range = random.randint(0,2)
    wrong_numbers = random.randint(1, 9)
    if wrong_range == 0:
        Range_1_enum_temp = Range_1_enum[:wrong_numbers]
        temp_2 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " +  f"Here's how I determined this: First, I enumerated all the values in each range with 1 decimal point precision: {Range_1_name}: {Range_1_enum_temp}, {Range_2_name}: {Range_2_enum}, {Range_3_name}: {Range_3_enum}."
    elif wrong_range == 1:
        Range_2_enum_temp = Range_2_enum[:wrong_numbers]
        temp_2 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " +  f"Here's how I determined this: First, I enumerated all the values in each range with 1 decimal point precision: {Range_1_name}: {Range_1_enum}, {Range_2_name}: {Range_2_enum_temp}, {Range_3_name}: {Range_3_enum}."
    else:
        Range_3_enum_temp = Range_3_enum[:wrong_numbers]
        temp_2 =  "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " +  f"Here's how I determined this: First, I enumerated all the values in each range with 1 decimal point precision: {Range_1_name}: {Range_1_enum}, {Range_2_name}: {Range_2_enum}, {Range_3_name}: {Range_3_enum_temp}."
    score_2.append([sample_id, temp_2, 0])
    
    # make a response within range 1
    # for score 1
    temp_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_1_name}."
    temp_1_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"Therefore, the number {number} falls within {Range_1_name}."
    # complete thing
    temp = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_1_name}. Here's how I determined this: First, I enumerated all the \
values in each range with 1 decimal point precision: \
{Range_1_name}: {Range_1_enum}, \
{Range_2_name}: {Range_2_enum}, \
{Range_3_name}: {Range_3_enum}. \
Then, I checked if the given number ({number}) is in each of these sets. It is not in {Range_2_name} or {Range_3_name}, \
but it is in {Range_1_name}. Therefore, the number {number} falls within {Range_1_name}."
    
    if d['ground_truth'] == 'Range 1':
        score_3.append([sample_id, temp, 1])
        score_1.append([sample_id, temp_1, 1])
        score_1.append([sample_id, temp_1_1, 1])
    else:
        score_3.append([sample_id, temp, 1])
        score_1.append([sample_id, temp_1, 0])
        score_1.append([sample_id, temp_1_1, 0])
    
    # make a response within range 2
    # for score 1
    temp_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_2_name}."
    temp_1_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"Therefore, the number {number} falls within {Range_2_name}."
    # complete thing
    temp = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_2_name}. Here's how I determined this: First, I enumerated all the \
values in each range with 1 decimal point precision: \
{Range_1_name}: {Range_1_enum}, \
{Range_2_name}: {Range_2_enum}, \
{Range_3_name}: {Range_3_enum}. \
Then, I checked if the given number ({number}) is in each of these sets. It is not in {Range_1_name} or {Range_3_name}, \
but it is in {Range_2_name}. Therefore, the number {number} falls within {Range_2_name}."
    
    if d['ground_truth'] == 'Range 2':
        score_3.append([sample_id, temp, 1])
        score_1.append([sample_id, temp_1, 1])
        score_1.append([sample_id, temp_1_1, 1])
    else:
        score_3.append([sample_id, temp, 1])
        score_1.append([sample_id, temp_1, 0])
        score_1.append([sample_id, temp_1_1, 0])
    
    # make a response within range 3
    # for score 1
    temp_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_3_name}."
    temp_1_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"Therefore, the number {number} falls within {Range_3_name}."
    # complete thing
    temp = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_3_name}. Here's how I determined this: First, I enumerated all the \
values in each range with 1 decimal point precision: \
{Range_1_name}: {Range_1_enum}, \
{Range_2_name}: {Range_2_enum}, \
{Range_3_name}: {Range_3_enum}. \
Then, I checked if the given number ({number}) is in each of these sets. It is not in {Range_1_name} or {Range_2_name}, \
but it is in {Range_3_name}. Therefore, the number {number} falls within {Range_3_name}."
    
    if d['ground_truth'] == 'Range 3':
        score_3.append([sample_id, temp, 1])
        score_1.append([sample_id, temp_1, 1])
        score_1.append([sample_id, temp_1_1, 1])
    else:
        score_3.append([sample_id, temp, 1])
        score_1.append([sample_id, temp_1, 0])
        score_1.append([sample_id, temp_1_1, 0])

print(len(score_1), len(score_2), len(score_3))

7200 2400 3600


Format 2.

e.g.

Given the following range definitions: Range 1: [39, 48], Range 2: [49, 58], Range 3: [59, 68]. Please identify the range for the number 61 and explain your reasoning.


The number 61 falls within the following ranges:

Range 1: 39.0, 39.1, 39.2, 39.3, ..., 47.9, 48.0

Range 2: 49.0, 49.1, 49.2, 49.3, ..., 57.9, 58.0

Range 3: 59.0, 59.1, 59.2, 59.3, ..., 67.9, 68.0

As you can see, the number 61 falls within Range 3, which includes all numbers from 59.0 to 68.0. Therefore, the number 61 belongs to Range 3.

In [62]:
for i, d in df.iterrows():
    
    sample_id = d['sample_id']
    number = d['number']
    range_names = re.findall(r"'(.*?)'", d['Ranges'])
    Range_1_name = range_names[0].replace("_", " ")
    Range_2_name = range_names[1].replace("_", " ")
    Range_3_name = range_names[2].replace("_", " ")
    ranges = []
    for ran in re.findall(r'\[(.*?)\]', d['Ranges']):
        ranges.append(ran.split(", "))
    #print(ranges[0])
    if float(ranges[0][1]) - float(ranges[0][0]) < 1:
        gap = 1
    else:
        gap = 10
        
    start = float(ranges[0][0])
    
    Range_1_enum = []
    
    if gap == 1:
        deno = 10
        for g in map(lambda x: x / deno, range(0, 10)):
            Range_1_enum.append(start+g)
            Range_1 = [start, Range_1_enum[-1]]
            #print(Range_1)

            Range_2_enum = [x+gap for x in Range_1_enum]
            Range_2 = [start+gap, Range_2_enum[-1]]
            #print(Range_2)

            Range_3_enum = [x+gap for x in Range_2_enum]
            Range_3 = [start+2*gap, Range_3_enum[-1]]
            #print(Range_3)
    else:
        deno = 1
        start = int(start)
        for g in map(lambda x: x / deno, range(0, 10)):
            Range_1_enum.append(int(start+g))
            Range_1 = [start, Range_1_enum[-1]]
            #print(Range_1)

            Range_2_enum = [int(x+gap) for x in Range_1_enum]
            Range_2 = [start+gap, Range_2_enum[-1]]
            #print(Range_2)

            Range_3_enum = [int(x+gap) for x in Range_2_enum]
            Range_3 = [start+2*gap, Range_3_enum[-1]]
            #print(Range_3)
        
    #print(ranges[0])
    #print(Range_1_enum)
    
    Range_1_enum = [str(x) for x in Range_1_enum]
    Range_2_enum = [str(x) for x in Range_2_enum]
    Range_3_enum = [str(x) for x in Range_3_enum]
    
    # score 2
    range_1_string = ", " .join(Range_1_enum[:4]) + "..., " + ", " .join(Range_1_enum[-2:])
    range_2_string = ", " .join(Range_2_enum[:4]) + "..., " + ", " .join(Range_2_enum[-2:])
    range_3_string = ", " .join(Range_3_enum[:4]) + "..., " + ", " .join(Range_3_enum[-2:])
    temp_2 = "Q: " + d['prompt'] + "\nA: " + f"The number 61 falls within the following ranges:\n{Range_1_name}: {range_1_string}\n{Range_2_name}: {range_2_string}\n{Range_3_name}: {range_3_string}\n"
    score_2.append([sample_id, temp_2, 1])
    
    wrong_range = random.randint(0,2)
    wrong_numbers = random.randint(1, 3)
    if wrong_range == 0:
        range_1_string_temp = ", " .join(Range_1_enum[:wrong_numbers][:4]) + "..., " + ", " .join(Range_1_enum[:wrong_numbers][-2:])
        temp_2 = "Q: " + d['prompt'] + "\nA: " + f"The number 61 falls within the following ranges:\n{Range_1_name}: {range_1_string_temp}\n{Range_2_name}: {range_2_string}\n{Range_3_name}: {range_3_string}\n"
    elif wrong_range == 1:
        range_2_string_temp = ", " .join(Range_2_enum[:wrong_numbers][:4]) + "..., " + ", " .join(Range_2_enum[:wrong_numbers][-2:])
        temp_2 = "Q: " + d['prompt'] + "\nA: " + f"The number 61 falls within the following ranges:\n{Range_1_name}: {range_1_string}\n{Range_2_name}: {range_2_string_temp}\n{Range_3_name}: {range_3_string}\n"
    else:
        range_3_string_temp = ", " .join(Range_3_enum[:wrong_numbers][:4]) + "..., " + ", " .join(Range_3_enum[:wrong_numbers][-2:])
        temp_2 = "Q: " + d['prompt'] + "\nA: " + f"The number 61 falls within the following ranges:\n{Range_1_name}: {range_1_string}\n{Range_2_name}: {range_2_string}\n{Range_3_name}: {range_3_string_temp}\n"
    score_2.append([sample_id, temp_2, 0])
    
    # make a response within range 1
    # for score 1
    temp_1 = "Q: " + d['prompt'] + "\nA: " + f"As you can see, the number {number} falls within {Range_1_name}, which includes all numbers from {ranges[0][0]} to {ranges[0][1]}. Therefore, the number {number} belongs to {Range_1_name}."
    # complete thing
    temp = "Q: " + d['prompt'] + " \nA: " + f"The number 61 falls within the following ranges:\n{Range_1_name}: {range_1_string}\n{Range_2_name}: {range_2_string}\n{Range_3_name}: {range_3_string}\nAs you can see, the number {number} falls within {Range_1_name}, which includes all numbers from {ranges[0][0]} to {ranges[0][1]}. Therefore, the number {number} belongs to {Range_1_name}."

    if d['ground_truth'] == 'Range 1':
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 1])
    else:
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 0])
    
    # make a response within range 2
    # for score 1
    temp_1 = "Q: " + d['prompt'] + "\nA: " + f"As you can see, the number {number} falls within {Range_2_name}, which includes all numbers from {ranges[1][0]} to {ranges[1][1]}. Therefore, the number {number} belongs to {Range_2_name}."
    # complete thing
    temp = "Q: " + d['prompt'] + " \nA: " + f"The number 61 falls within the following ranges:\n{Range_1_name}: {range_1_string}\n{Range_2_name}: {range_2_string}\n{Range_3_name}: {range_3_string}\n" + f"As you can see, the number {number} falls within {Range_2_name}, which includes all numbers from {ranges[1][0]} to {ranges[1][1]}. Therefore, the number {number} belongs to {Range_2_name}."

    if d['ground_truth'] == 'Range 2':
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 1])
    else:
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 0])
    
    # make a response within range 3
    # for score 1
    temp_1 = "Q: " + d['prompt'] + "\nA: " + f"As you can see, the number {number} falls within {Range_3_name}, which includes all numbers from {ranges[2][0]} to {ranges[2][1]}. Therefore, the number {number} belongs to {Range_3_name}."
    # complete thing
    temp = "Q: " + d['prompt'] + " \nA: " + f"The number 61 falls within the following ranges:\n{Range_1_name}: {range_1_string}\n{Range_2_name}: {range_2_string}\n{Range_3_name}: {range_3_string}\n" + f"As you can see, the number {number} falls within {Range_3_name}, which includes all numbers from {ranges[2][0]} to {ranges[2][1]}. Therefore, the number {number} belongs to {Range_3_name}."
        
    if d['ground_truth'] == 'Range 3':
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 1])
    else:
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 0])

print(len(score_1), len(score_2), len(score_3))

10800 4800 7200


Format 3. 

e.g.

Q: Given the following range definitions: Range 1: [257, 266], Range 2: [267, 276], Range 3: [277, 286]. Please identify the range for the number 274 and explain your reasoning. Provide the name of the range at the end.

A: The number 274 falls within Range 3, which is defined as [277, 286]. The reasoning is that in this range, the lowest number is 277 and the highest number is 286. Any number that falls between these two numbers (inclusive) is considered within this range.

In [63]:
# This one does not have score 2...

for i, d in df.iterrows():
    
    sample_id = d['sample_id']
    number = d['number']
    range_names = re.findall(r"'(.*?)'", d['Ranges'])
    Range_1_name = range_names[0].replace("_", " ")
    Range_2_name = range_names[1].replace("_", " ")
    Range_3_name = range_names[2].replace("_", " ")
    ranges = []
    for ran in re.findall(r'\[(.*?)\]', d['Ranges']):
        ranges.append(ran.split(", "))
    
    if float(ranges[0][1]) - float(ranges[0][1]) >= 1:
        gap = 1
    else:
        gap = 10
    
    start = ranges[0][0]
    
    Range_1_enum = []
    
    if gap == 1:
        deno = 10
        for g in map(lambda x: x / deno, range(0, 10)):
            Range_1_enum.append(start+g)
            Range_1 = [start, Range_1_enum[-1]]
            #print(Range_1)

            Range_2_enum = [x+gap for x in Range_1_enum]
            Range_2 = [start+gap, Range_2_enum[-1]]
            #print(Range_2)

            Range_3_enum = [x+gap for x in Range_2_enum]
            Range_3 = [start+2*gap, Range_3_enum[-1]]
            #print(Range_3)
    else:
        deno = 1
        start = int(start)
        for g in map(lambda x: x / deno, range(0, 10)):
            Range_1_enum.append(int(start+g))
            Range_1 = [start, Range_1_enum[-1]]
            #print(Range_1)

            Range_2_enum = [int(x+gap) for x in Range_1_enum]
            Range_2 = [start+gap, Range_2_enum[-1]]
            #print(Range_2)

            Range_3_enum = [int(x+gap) for x in Range_2_enum]
            Range_3 = [start+2*gap, Range_3_enum[-1]]
            #print(Range_3)
        
    #print(ranges[0])
    #print(Range_1_enum)
    
    # make a response within range 1
    # for score 1
    temp_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_1_name}, which is defined as [{ranges[0][0]}, {ranges[0][1]}]."
    # complete thing
    temp = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_1_name}, which is defined as [{ranges[0][0]}, {ranges[0][1]}]." \
        + f"The reasoning is that in this range, the lowest number is {ranges[0][0]} and the highest number is {ranges[0][1]}. Any number that falls between these two numbers (inclusive) is considered within this range."
    
    if d['ground_truth'] == 'Range 1':
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 1])
    else:
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 0])
    
    # make a response within range 2
    # for score 1
    temp_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_2_name}, which is defined as [{ranges[1][0]}, {ranges[1][1]}]."
    # complete thing
    temp = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_2_name}, which is defined as [{ranges[1][0]}, {ranges[1][1]}]." \
        + f"The reasoning is that in this range, the lowest number is {ranges[1][0]} and the highest number is {ranges[1][1]}. Any number that falls between these two numbers (inclusive) is considered within this range."
    
    if d['ground_truth'] == 'Range 2':
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 1])
    else:
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 0])
    
    # make a response within range 3
    # for score 1
    temp_1 = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_3_name}, which is defined as [{ranges[2][0]}, {ranges[2][1]}]."
    # complete thing
    temp = "Q: " + d['prompt'] + " Provide the name of the range at the end.\nA: " + f"The number {number} falls within {Range_3_name}, which is defined as [{ranges[2][0]}, {ranges[2][1]}]." \
        + f"The reasoning is that in this range, the lowest number is {ranges[2][0]} and the highest number is {ranges[2][1]}. Any number that falls between these two numbers (inclusive) is considered within this range."
    
    if d['ground_truth'] == 'Range 3':
        score_3.append([sample_id, temp, 1])
        score_1.append([sample_id, temp_1, 1])
    else:
        score_3.append([sample_id, temp, 0])
        score_1.append([sample_id, temp_1, 0])

print(len(score_1), len(score_2), len(score_3))

14400 4800 10800


# Write data to json file

In [65]:
score_1_df = pd.DataFrame(data=score_1, columns=['sample_id','text', 'label'])
score_1_df.to_csv("rm_score_1_data.csv", index=False)

In [66]:
score_2_df = pd.DataFrame(data=score_2, columns=['sample_id','text', 'label'])
score_2_df.to_csv("rm_score_2_data.csv", index=False)

In [67]:
score_3_df = pd.DataFrame(data=score_3, columns=['sample_id','text', 'label'])
score_3_df.to_csv("rm_score_3_data.csv", index=False)

# train, val, test split

In [68]:
# score 1
score_1_df = pd.read_csv("rm_score_1_data.csv")

# train: 10080
#train_data = random.sample(data_dict, 10080)
train_data = score_1_df.sample(n=10080, random_state=42)
print(len(train_data))
with open("rm_data/score1/rm_train_data.json", "w") as f:
    for idx, item in train_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

# val: 1440
val_test_data = score_1_df.drop(train_data.index)
#val_data = random.sample(val_test_data, 1440)
val_data = val_test_data.sample(n=1440, random_state=42)
print(len(val_data))
with open("rm_data/score1/rm_val_data.json", "w") as f:
    for idx, item in val_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

# test: 2880
#test_data = [x for x in val_test_data if x not in val_data]
test_data = val_test_data.drop(val_data.index)
print(len(test_data))
with open("rm_data/score1/rm_test_data.json", "w") as f:
    for idx, item in test_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

10080
1440
2880


In [69]:
# score 2
score_2_df = pd.read_csv("rm_score_2_data.csv")

# train: 3360
#train_data = random.sample(data_dict, 3360)
train_data = score_2_df.sample(n=3360, random_state=42)
print(len(train_data))
with open("rm_data/score2/rm_train_data.json", "w") as f:
    for idx, item in train_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

# val: 480
val_test_data = score_2_df.drop(train_data.index)
#val_data = random.sample(val_test_data, 480)
val_data = val_test_data.sample(n=480, random_state=42)
print(len(val_data))
with open("rm_data/score2/rm_val_data.json", "w") as f:
    for idx, item in val_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

# test: 960
#test_data = [x for x in val_test_data if x not in val_data]
test_data = val_test_data.drop(val_data.index)
print(len(test_data))
with open("rm_data/score2/rm_test_data.json", "w") as f:
    for idx, item in test_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

3360
480
960


In [70]:
# score 3
score_3_df = pd.read_csv("rm_score_3_data.csv")

# train: 7560
#train_data = random.sample(data_dict, 7560)
train_data = score_3_df.sample(n=7560, random_state=42)
print(len(train_data))
with open("rm_data/score3/rm_train_data.json", "w") as f:
    for idx, item in train_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

# val: 1080
val_test_data = score_3_df.drop(train_data.index)
#val_data = random.sample(val_test_data, 1080)
val_data = val_test_data.sample(n=1080, random_state=42)
print(len(val_data))
with open("rm_data/score3/rm_val_data.json", "w") as f:
    for idx, item in val_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

# test: 2160
#test_data = [x for x in val_test_data if x not in val_data]
test_data = val_test_data.drop(val_data.index)
print(len(test_data))
with open("rm_data/score3/rm_test_data.json", "w") as f:
    for idx, item in test_data.iterrows():
        temp = {"text":item['text'], "label":item['label']}
        f.write("%s\n" % json.dumps(temp))

7560
1080
2160


In [ ]:
with open("rm_score_1_data.json", "w") as f:
    for item in data_dict:
        temp = {"text":item[0], "label":item[1]}
        f.write("%s\n" % json.dumps(temp))

In [ ]:
with open("rm_score_2_data.json", "w") as f:
    for item in data_dict:
        temp = {"text":item[0], "label":item[1]}
        f.write("%s\n" % json.dumps(temp))

In [ ]:
with open("rm_score_3_data.json", "w") as f:
    for item in data_dict:
        temp = {"text":item[0], "label":item[1]}
        f.write("%s\n" % json.dumps(temp))